# 0/1 Knapsack Problem

The goal is to maximize the total value of items that can be put into a knapsack with a given capacity. Each item has a weight and a value, and you can either take the whole item or leave it.

Input:
- `capacity`: the maximum weight the knapsack can hold.
- `weights`: a list of weights of the items.
- `values`: a list of values corresponding to the items.

Optimal solution:
- **Time Complexity**: O(n * capacity)
- **Space Complexity**: O(n * capacity)

where n is the number of items.

In [1]:
import json
from typing import List

In [2]:
from utils.test_runner import run_tests

## Test Cases

In [3]:
test_cases = [
    {
        "params": {
            "values": [1,6,10,16],
            "weights": [1,2,3,5],
            "capacity": 0,
        },
        "expected": 0,
    },
    {
        "params": {
            "values": [1,6,10,16],
            "weights": [1,2,3,5],
            "capacity": 1,
        },
        "expected": 1,
    },
    {
        "params": {
            "values": [1,6,10,16],
            "weights": [1,2,3,5],
            "capacity": 2,
        },
        "expected": 6,
    },
    {
        "params": {
            "values": [1,6,10,16],
            "weights": [1,2,3,5],
            "capacity": 3,
        },
        "expected": 10,
    },
    {
        "params": {
            "values": [1,6,10,16],
            "weights": [1,2,3,5],
            "capacity": 4,
        },
        "expected": 11,
    },
    {
        "params": {
            "values": [1,6,10,16],
            "weights": [1,2,3,5],
            "capacity": 5,
        },
        "expected": 16,
    },
    {
        "params": {
            "values": [1,6,10,16],
            "weights": [1,2,3,5],
            "capacity": 6,
        },
        "expected": 17,
    },
    {
        "params": {
            "values": [1,6,10,16],
            "weights": [1,2,3,5],
            "capacity": 7,
        },
        "expected": 22,
    },
    {
        "params": {
            "values": [1,6,10,16],
            "weights": [1,2,3,5],
            "capacity": 8,
        },
        "expected": 26,
    },
    {
        "params": {
            "values": [1,6,10,16],
            "weights": [1,2,3,5],
            "capacity": 9,
        },
        "expected": 27,
    },
    {
        "params": {
            "values": [1,6,10,16],
            "weights": [1,2,3,5],
            "capacity": 10,
        },
        "expected": 32,
    },
    {
        "params": {
            "values": [1,6,10,16],
            "weights": [1,2,3,5],
            "capacity": 11,
        },
        "expected": 33,
    },
    {
        "params": {
            "values": [24, 13, 23, 15, 16, 20, 18, 12, 17, 19, 22, 14, 21, 11, 25, 10, 26, 27, 28, 29],
            "weights": [12, 7, 11, 8, 9, 10, 13, 6, 7, 8, 9, 5, 10, 4, 11, 3, 12, 13, 14, 15],
            "capacity": 50,
        },
        "expected": 122,
    },
]

## Recursive Unoptimized

Let f(C, N) be the optimal value in the knapsack of capacity C and values v[1..N]

Now, two cases:
1. item[N] is not part of the optimal solution
2. item[N] is part of the optimal solution

Thus
1. f(C, N) = f(C, N-1)
2. f(C, N) = v[N] + f(C-v[N], N-1)

Therefore, f(C, N) = max(f(C, N-1), f(C-w[N], N-1))

Base case:
f(0, N) = f(C, 0) = 0

In [4]:
def knapsack_recursive(values: List[int], weights: List[int], capacity: int) -> int:
    assert len(values) == len(weights), "'values' and 'weights' must have the same size"
    stats = {'calls': 0}

    def _knapsack_recursive(capacity: int, n: int) -> int:
        nonlocal stats
        stats['calls'] += 1
        if capacity <= 0 or n == 0:
            return 0
        if capacity < weights[n - 1]:
            return _knapsack_recursive(capacity, n - 1)
        return max(
            _knapsack_recursive(capacity, n - 1),
            values[n - 1] + _knapsack_recursive(capacity - weights[n - 1], n - 1)
        )

    ans = _knapsack_recursive(capacity, len(values))
    print(f"stats: {json.dumps(stats)}")
    return ans

**Time Complexity:**  
The `knapsack_recursive` function explores all possible subsets of items. For each item, there are two choices: include or exclude. Therefore, the total number of recursive calls is $O(2^n)$, where $n$ is the number of items.

**Space Complexity:**  
The maximum depth of the recursion tree is $n$ (the number of items), so the space complexity due to the call stack is $O(n)$.


In [5]:
run_tests(test_cases, knapsack_recursive)

stats: {"calls": 1}
stats: {"calls": 6}
stats: {"calls": 7}
stats: {"calls": 10}
stats: {"calls": 13}
stats: {"calls": 15}
stats: {"calls": 21}
stats: {"calls": 22}
stats: {"calls": 25}
stats: {"calls": 28}
stats: {"calls": 29}
stats: {"calls": 31}
stats: {"calls": 91420}
Passed 13/13 tests.


## Top-Down with Memoization

In [6]:
def knapsack_memoized(values: List[int], weights: List[int], capacity: int) -> int:
    assert len(values) == len(weights), "'values' and 'weights' must have the same size"
    stats = {'calls': 0, 'hits': 0, 'misses': 0}
    cache = {}

    def _knapsack_memoized(capacity: int, n: int) -> int:
        nonlocal stats, cache
        stats['calls'] += 1
        if (capacity, n) in cache:
            stats['hits'] += 1
        elif capacity <= 0 or n <= 0:
            stats['misses'] += 1
            cache[(capacity, n)] = 0
        elif capacity < weights[n - 1]:
            stats['misses'] += 1
            cache[(capacity, n)] = _knapsack_memoized(capacity, n - 1)
        else:
            stats['misses'] += 1
            cache[(capacity, n)] = max(
                _knapsack_memoized(capacity, n - 1),
                values[n - 1] + _knapsack_memoized(capacity - weights[n - 1], n - 1)
            )
        return cache[(capacity, n)]

    ans = _knapsack_memoized(capacity, len(values))
    print(f"stats: {json.dumps(stats)}")
    return ans

In [7]:
run_tests(test_cases, knapsack_memoized)

stats: {"calls": 1, "hits": 0, "misses": 1}
stats: {"calls": 6, "hits": 0, "misses": 6}
stats: {"calls": 7, "hits": 0, "misses": 7}
stats: {"calls": 10, "hits": 0, "misses": 10}
stats: {"calls": 13, "hits": 1, "misses": 12}
stats: {"calls": 15, "hits": 1, "misses": 14}
stats: {"calls": 19, "hits": 2, "misses": 17}
stats: {"calls": 20, "hits": 2, "misses": 18}
stats: {"calls": 23, "hits": 2, "misses": 21}
stats: {"calls": 26, "hits": 3, "misses": 23}
stats: {"calls": 27, "hits": 3, "misses": 24}
stats: {"calls": 29, "hits": 3, "misses": 26}
stats: {"calls": 1280, "hits": 524, "misses": 756}
Passed 13/13 tests.


## Bottom-Up Dynamic Programming

In [8]:
def knapsack_dp(values: List[int], weights: List[int], capacity: int) -> int:
    assert len(values) == len(weights), "'values' and 'weights' should have the same size"
    N = len(values)
    dp = [[0 for _ in range(N + 1)] for _ in range(capacity + 1)]
    for c in range(1, capacity + 1):
        for n in range(1, N + 1):
            dp[c][n] = max(
                dp[c][n - 1],
                values[n - 1] + dp[c - weights[n - 1]][n - 1] if c >= weights[n - 1] else 0
            )
    stats = {'cells': N * capacity}
    print(f"stats: {json.dumps(stats)}")
    return dp[capacity][N]


In [9]:
run_tests(test_cases, knapsack_dp)

stats: {"cells": 0}
stats: {"cells": 4}
stats: {"cells": 8}
stats: {"cells": 12}
stats: {"cells": 16}
stats: {"cells": 20}
stats: {"cells": 24}
stats: {"cells": 28}
stats: {"cells": 32}
stats: {"cells": 36}
stats: {"cells": 40}
stats: {"cells": 44}
stats: {"cells": 1000}
Passed 13/13 tests.
